In [2]:
from dfply import *
import pandas as pd
import numpy as np

In [19]:
surveys = pd.read_csv('./health_survey.csv') >> rename(participant = 'Unnamed: 0')
surveys >> head()

,participant,F1,F5,F2,F1.1,F2.1,F6,F4,F3,F5.1,...,F2.9,F3.4,F4.3,F2.10,F1.7,F6.4,F4.4,F5.7,F3.5,F2.11
0,1,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,...,Somewhat Agree,Somewhat Disagree,Neither Agree nor Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Agree
1,2,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Neither Agree nor Disagree,Neither Agree nor Disagree,...,Somewhat Agree,Somewhat Agree,Neither Agree nor Disagree,Somewhat Agree,Somewhat Agree,Somewhat Disagree,Neither Agree nor Disagree,Somewhat Agree,Neither Agree nor Disagree,Somewhat Agree
2,3,Strongly Agree,Neither Agree nor Disagree,Somewhat Agree,Strongly Agree,Strongly Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,...,Somewhat Agree,Somewhat Agree,Neither Agree nor Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Strongly Agree,Strongly Disagree,Somewhat Agree
3,4,Somewhat Agree,Somewhat Agree,Strongly Agree,Somewhat Agree,Strongly Agree,Neither Agree nor Disagree,Neither Agree nor Disagree,Somewhat Disagree,Somewhat Agree,...,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Neither Agree nor Disagree,Neither Agree nor Disagree,Neither Agree nor Disagree,Somewhat Agree,Somewhat Disagree,Somewhat Agree
4,5,Strongly Agree,Strongly Disagree,Neither Agree nor Disagree,Strongly Agree,Somewhat Agree,Strongly Disagree,Strongly Agree,Somewhat Agree,Neither Agree nor Disagree,...,Somewhat Agree,Somewhat Agree,Neither Agree nor Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Strongly Agree,Somewhat Disagree,Somewhat Agree


In [80]:
reverse = (pd.read_csv('./ReverseCodingItems.csv') >> 
           drop(columns_to("Needs Reverse Coding?")) >>
           rename(needs_reverse = 'Needs Reverse Coding?',
                  question = 'Column Name'))
reverse.head()

,needs_reverse,question
0,No,F1
1,Yes,F5
2,No,F2
3,No,F1.1
4,No,F2.1


In [81]:
reg_coding = {'Strongly Agree':5,
              'Somewhat Agree':4,
              'Neither Agree nor Disagree':3,
              'Somewhat Disagree':2,
              'Strongly Disagree':1}
rev_coding = {resp: 6 - val for resp, val in reg_coding.items()}
ind_coding = {'Yes': 1, 'No':0}

In [85]:
df = (surveys >> 
      gather('question', 
             'response', 
             surveys.columns[1:]) >>
      left_join(reverse, 
                by='question') >>
      mutate(question_type = X.question.str.slice(0,2),
             reg_code = X.response.replace(reg_coding),
             rev_code = X.response.replace(rev_coding),
             rev_ind = X.needs_reverse.replace(ind_coding)) >>
      mutate(coded_value = X.rev_ind*X.rev_code + (1 - X.rev_ind)*X.reg_code)>>
      group_by(X.participant, 
               X.question_type) >>
      summarise(total = X.coded_value.sum()) >>
      spread(X.question_type, X.total))
df.head()

,participant,F1,F2,F3,F4,F5,F6
0,1,31.0,48.0,20.0,17.0,28.0,18.0
1,2,31.0,47.0,19.0,17.0,27.0,20.0
2,3,36.0,46.0,19.0,18.0,32.0,17.0
3,4,32.0,54.0,12.0,15.0,30.0,16.0
4,5,37.0,47.0,22.0,19.0,36.0,19.0
